In [1]:
import joblib
import re 
import nltk
import pandas as pd
import numpy as np
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords, words
from sklearn.metrics import accuracy_score
nltk.download('words')
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package words to
[nltk_data]     C:\Users\francisco.neto\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\francisco.neto\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\francisco.neto\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\francisco.neto\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
df = pd.read_csv('src/test_binary.csv')

In [3]:
df.head()

,id,comment_text,Toxic
0,0001ea8717f6de06,Thank you for understanding. I think very high...,Non-Toxic
1,000247e83dcc1211,:Dear god this site is horrible.,Non-Toxic
2,0002f87b16116a7f,"""::: Somebody will invariably try to add Relig...",Non-Toxic
3,0003e1cccfd5a40a,""" \n\n It says it right there that it IS a typ...",Non-Toxic
4,00059ace3e3e9a53,""" \n\n == Before adding a new product to the l...",Non-Toxic


In [4]:
X_test = df['comment_text']

In [5]:
y_test = df['Toxic']

In [6]:
substitute = {
    "< br />":'',
    '<br />':'',
    "'ll":' will',
    "'s":' is',
    "'re":' are',
    "n't":' not'
}

In [7]:
def preprocessing(X, dict_change):
    new_x = []
    for i in X:
        text_lower = i.lower()
        for key in dict_change:
            if key in text_lower:
                text_lower.replace(key, dict_change[key])
        text_lower = re.sub("\W", ' ', text_lower)
        text_lower = re.sub(" +", ' ', text_lower)
        tokens = nltk.word_tokenize(text_lower)
        tokens = [x for x in tokens if x not in stopwords.words('english')]  
        new_x.append(' '.join(tokens))
    return new_x

def lemmatizer(X):
    lemmatizer = WordNetLemmatizer()
    new_x = []
    for i in X:
        tokens = nltk.word_tokenize(i)
        for j in range(len(tokens)):
            tokens[j] = lemmatizer.lemmatize(tokens[j])
        new_x.append(' '.join(tokens))
    return new_x


In [10]:
X_test = preprocessing(X_test, substitute)

In [11]:
X_test = lemmatizer(X_test)

In [12]:
X_test[0]

'thank understanding think highly would revert without discussion'

In [13]:
transformer_load = joblib.load("transformer/transformer.pkl")

In [14]:
X_test = transformer_load.transform(X_test)

In [15]:
model_load = joblib.load('models/model.pkl')

In [16]:
Y_pred = model_load.predict(X_test)

In [23]:
lista = []
for i in Y_pred:
    lista.append(i)

In [27]:
df.insert(3,'Predict',lista)

In [35]:
df.head(5)

,id,comment_text,Toxic,Predict
0,0001ea8717f6de06,Thank you for understanding. I think very high...,Non-Toxic,Non-Toxic
1,000247e83dcc1211,:Dear god this site is horrible.,Non-Toxic,Toxic
2,0002f87b16116a7f,"""::: Somebody will invariably try to add Relig...",Non-Toxic,Toxic
3,0003e1cccfd5a40a,""" \n\n It says it right there that it IS a typ...",Non-Toxic,Non-Toxic
4,00059ace3e3e9a53,""" \n\n == Before adding a new product to the l...",Non-Toxic,Non-Toxic


In [38]:
df.to_csv("src/dataset_results.csv",sep="\t")

In [30]:
accuracy_score(y_test,Y_pred)

0.6048798024320861

# Analysis

The accuracy of this experiment may have been low, at the expense of factors such as the amount of data much higher than the training datasets and also because the model had an average accuracy of 80% and in this case it was 60%, perhaps because of the fact the model has been trained with another dataset.